# Phase 5: P.1812 Loss Calculation

## Overview
This notebook demonstrates Phase 5 of the radio propagation pipeline:
- Load CSV profiles generated by Phase 4
- Process each profile using ITU-R P.1812-6 `bt_loss()` function
- Calculate basic transmission loss (Lb) and electric field strength (Ep)
- Store results in output DataFrame and/or CSV

## Inputs
- CSV profiles from Phase 4: `data/input/profiles/*.csv`

## Outputs
- DataFrame with loss/field strength results
- CSV export: `data/output/spreadsheets/p1812_results.csv`

## 1. Setup and Imports

In [ ]:
import ast
import csv
import time
from pathlib import Path
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd

# P.1812-6 propagation model
import Py1812.P1812

print("✓ Imports successful")

## 2. Locate Project Root and Data Paths

In [ ]:
# Dynamic path detection
current_dir = Path.cwd()

# Search for project root (contains src/mst_gis or config_sentinel_hub.py)
project_root = None
for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'src' / 'mst_gis').exists() or (parent / 'config_sentinel_hub.py').exists():
        project_root = parent
        break

if not project_root:
    raise RuntimeError(
        f"Could not locate project root from {current_dir}. "
        "Expected src/mst_gis or config_sentinel_hub.py in parent directories."
    )

# Define data paths
profiles_dir = project_root / 'data' / 'input' / 'profiles'
output_dir = project_root / 'data' / 'output' / 'spreadsheets'

print(f"Project root: {project_root}")
print(f"Profiles dir: {profiles_dir}")
print(f"Output dir: {output_dir}")

# Create output directory if needed
output_dir.mkdir(parents=True, exist_ok=True)

## 3. Load CSV Profiles

Each row represents one azimuth profile with:
- Frequency, time percentage
- Distance, height, resistance, category, zone arrays
- TX/RX locations and antenna heights

In [ ]:
def load_profiles(profiles_dir: Path) -> List[List[str]]:
    """
    Load all CSV profiles from directory.
    
    Each profile is a semicolon-delimited row.
    Skip header rows.
    """
    profiles = []
    
    csv_files = list(profiles_dir.glob("*.csv"))
    print(f"Found {len(csv_files)} CSV files")
    
    for file in csv_files:
        print(f"  Loading: {file.name}")
        with file.open(newline="", encoding="utf-8") as f:
            # Skip header, read remaining rows
            rows = list(csv.reader(f, delimiter=";"))[1:]
            profiles.extend(rows)
            print(f"    -> {len(rows)} profiles")
    
    print(f"\nTotal profiles loaded: {len(profiles)}")
    return profiles

# Load profiles
profiles = load_profiles(profiles_dir)

## 4. Parse Profile Parameters

Convert CSV strings to numpy arrays and floats for P.1812 function.

In [ ]:
def process_loss_parameters(profile: List[str]) -> Tuple:
    """
    Parse CSV row into P.1812 bt_loss() parameters.
    
    Returns tuple of (f, p, d, h, R, Ct, zone, htg, hrg, pol, phi_t, phi_r, lam_t, lam_r)
    
    Where:
    - f: frequency (GHz)
    - p: time percentage (%)
    - d: distance array (km)
    - h: height array (m)
    - R: resistance array (ohms)
    - Ct: land cover category array
    - zone: zone ID array
    - htg: TX antenna height (m)
    - hrg: RX antenna height (m)
    - pol: polarization (1=horiz, 2=vert)
    - phi_t: TX latitude
    - phi_r: RX latitude
    - lam_t: TX longitude
    - lam_r: RX longitude
    """
    # Parse first 14 columns (skip azimuth at index 14)
    parameters = [ast.literal_eval(parameter) for parameter in profile[0:14]]
    
    return (
        float(parameters[0]),                                  # f: frequency
        float(parameters[1]),                                  # p: time percentage
        np.array([float(v) for v in parameters[2]]),          # d: distances
        np.array([float(v) for v in parameters[3]]),          # h: heights
        np.array([float(v) for v in parameters[4]]),          # R: resistances
        np.array([int(v) for v in parameters[5]]),            # Ct: land cover categories
        np.array([int(v) for v in parameters[6]]),            # zone: zone IDs
        float(parameters[7]),                                  # htg: TX antenna height
        float(parameters[8]),                                  # hrg: RX antenna height
        int(parameters[9]),                                    # pol: polarization
        float(parameters[10]),                                 # phi_t: TX latitude
        float(parameters[11]),                                 # phi_r: RX latitude
        float(parameters[12]),                                 # lam_t: TX longitude
        float(parameters[13]),                                 # lam_r: RX longitude
    )

# Test on first profile
if profiles:
    params = process_loss_parameters(profiles[0])
    print(f"Example profile parameters:")
    print(f"  Frequency: {params[0]} GHz")
    print(f"  Time %: {params[1]}%")
    print(f"  Distance array length: {len(params[2])}")
    print(f"  TX location: ({params[12]}, {params[10]})")
    print(f"  RX location: ({params[13]}, {params[11]})")

## 5. Execute P.1812 Loss Calculation

In [ ]:
def calculate_p1812_loss(parameters: Tuple) -> Tuple[float, float]:
    """
    Call P.1812 bt_loss() function.
    
    Returns (Lb, Ep) where:
    - Lb: Basic transmission loss (dB)
    - Ep: Electric field strength (dBμV/m)
    """
    try:
        Lb, Ep = Py1812.P1812.bt_loss(*parameters)
        return float(Lb), float(Ep)
    except Exception as e:
        print(f"  ⚠ Error calculating loss: {e}")
        return np.nan, np.nan

# Test on first profile
if profiles:
    params = process_loss_parameters(profiles[0])
    start = time.perf_counter()
    Lb, Ep = calculate_p1812_loss(params)
    elapsed = time.perf_counter() - start
    
    print(f"P.1812 calculation for profile 1:")
    print(f"  Basic transmission loss (Lb): {Lb:.2f} dB")
    print(f"  Electric field strength (Ep): {Ep:.2f} dBμV/m")
    print(f"  Calculation time: {elapsed:.4f} seconds")

## 6. Process All Profiles and Store Results

In [ ]:
def process_all_profiles(profiles: List[List[str]]) -> pd.DataFrame:
    """
    Process all profiles through P.1812 and collect results.
    
    Returns DataFrame with columns:
    - profile_id: Profile index
    - frequency_ghz: Frequency
    - time_percentage: Time percentage
    - tx_lat, tx_lon: Transmitter location
    - rx_lat, rx_lon: Receiver location (last point in profile)
    - max_distance_km: Maximum distance in profile
    - Lb_dB: Basic transmission loss
    - Ep_dBuV_m: Electric field strength
    """
    results = []
    failed_profiles = 0
    
    start_total = time.perf_counter()
    
    for idx, profile in enumerate(profiles):
        try:
            params = process_loss_parameters(profile)
            Lb, Ep = calculate_p1812_loss(params)
            
            # Extract metadata
            f, p, d, h, R, Ct, zone, htg, hrg, pol, phi_t, phi_r, lam_t, lam_r = params
            
            results.append({
                'profile_id': idx + 1,
                'frequency_ghz': f,
                'time_percentage': p,
                'polarization': pol,
                'tx_lat': phi_t,
                'tx_lon': lam_t,
                'rx_lat': phi_r,
                'rx_lon': lam_r,
                'antenna_height_tx_m': htg,
                'antenna_height_rx_m': hrg,
                'max_distance_km': float(d[-1]),
                'Lb_dB': Lb,
                'Ep_dBuV_m': Ep,
                'num_distance_points': len(d),
            })
            
            if (idx + 1) % 10 == 0:
                print(f"  Processed {idx + 1}/{len(profiles)} profiles...")
        
        except Exception as e:
            print(f"  ✗ Failed on profile {idx + 1}: {e}")
            failed_profiles += 1
            continue
    
    elapsed_total = time.perf_counter() - start_total
    
    print(f"\n✓ Processing complete")
    print(f"  Successful: {len(results)}/{len(profiles)}")
    print(f"  Failed: {failed_profiles}")
    print(f"  Total time: {elapsed_total:.2f} seconds")
    print(f"  Avg per profile: {elapsed_total/len(profiles):.4f} seconds")
    
    return pd.DataFrame(results)

# Process all profiles
results_df = process_all_profiles(profiles)

## 7. Examine Results

In [ ]:
print(f"Results shape: {results_df.shape}")
print(f"\nFirst 5 rows:")
print(results_df.head())
print(f"\nStatistics:")
print(results_df[['Lb_dB', 'Ep_dBuV_m', 'max_distance_km']].describe())

## 8. Export Results to CSV

In [ ]:
# Export to CSV
csv_path = output_dir / 'p1812_results.csv'
results_df.to_csv(csv_path, index=False)

print(f"✓ Exported results to:")
print(f"  {csv_path}")
print(f"  File size: {csv_path.stat().st_size / 1024:.1f} KB")

## 9. Visualization (Optional)

In [ ]:
import matplotlib.pyplot as plt

# Plot loss vs distance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Basic transmission loss distribution
ax1.hist(results_df['Lb_dB'], bins=30, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Basic Transmission Loss (dB)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Lb Values')
ax1.grid(alpha=0.3)

# Electric field strength distribution
ax2.hist(results_df['Ep_dBuV_m'], bins=30, edgecolor='black', alpha=0.7, color='orange')
ax2.set_xlabel('Electric Field Strength (dBμV/m)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Ep Values')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Scatter: loss vs distance
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(results_df['max_distance_km'], results_df['Lb_dB'], alpha=0.6)
ax.set_xlabel('Maximum Distance (km)')
ax.set_ylabel('Basic Transmission Loss (dB)')
ax.set_title('Loss vs Distance')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Summary

In [ ]:
print("\n" + "="*60)
print("PHASE 5: P.1812 LOSS CALCULATION - SUMMARY")
print("="*60)
print(f"\nInput Profiles:")
print(f"  Location: {profiles_dir}")
print(f"  Count: {len(profiles)}")

print(f"\nResults:")
print(f"  Successful: {len(results_df)}")
print(f"  Failed: {len(profiles) - len(results_df)}")

print(f"\nBasic Transmission Loss (Lb):")
print(f"  Min: {results_df['Lb_dB'].min():.2f} dB")
print(f"  Max: {results_df['Lb_dB'].max():.2f} dB")
print(f"  Mean: {results_df['Lb_dB'].mean():.2f} dB")
print(f"  Std: {results_df['Lb_dB'].std():.2f} dB")

print(f"\nElectric Field Strength (Ep):")
print(f"  Min: {results_df['Ep_dBuV_m'].min():.2f} dBμV/m")
print(f"  Max: {results_df['Ep_dBuV_m'].max():.2f} dBμV/m")
print(f"  Mean: {results_df['Ep_dBuV_m'].mean():.2f} dBμV/m")
print(f"  Std: {results_df['Ep_dBuV_m'].std():.2f} dBμV/m")

print(f"\nOutput:")
print(f"  Location: {csv_path}")
print(f"  Rows: {len(results_df)}")
print(f"  Columns: {list(results_df.columns)}")
print("="*60)